In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os


In [48]:
#list names of all files
os.listdir('data')


['leads.csv', 'rates_land.csv', 'rates_sea.csv', 'rates_air.csv']

In [49]:
#read the data
list_of_files = os.listdir('data')
leads, air, land, sea = [pd.read_csv('data/' + file) for file in list_of_files]

In [50]:
leads


,request_id,request_date,valid_date_from,valid_date_to,weight,weight_unit,amount,container_type,shipping_type,hs_code,...,liquid_cargo,imo_code,city_from_country,city_from,port_from_country,port_from,port_to_country,port_to,city_to_country,city_to
0,855641,2023-09-05,2023-09-16,2023-09-23,26.63,Metric Ton,1.0,40' Refrigerated,FCL,06-14,...,False,NaN,NaN,NaN,ID,Jakarta,BD,Chittagong,NaN,NaN
1,855652,2023-09-05,2023-09-30,2023-10-07,28.00,Metric Ton,1.0,20' Standard,FCL,25-27,...,False,NaN,NaN,NaN,BR,Vitoria,US,Houston,US,"Houston, TX, USA"
2,855664,2023-09-05,2023-09-05,2023-09-12,200.00,Metric Ton,1.0,20' Standard,FCL,NaN,...,False,NaN,NaN,NaN,CN,Chengdu,DO,Caucedo,DO,Dominican Republic
3,855689,2023-09-05,2023-09-30,2023-10-07,26.00,Metric Ton,20.0,40' Standard,FCL,0910,...,False,NaN,NaN,NaN,NG,Apapa,AE,Jebel Ali,AE,Mina Jebel Ali - Dubai - United Arab Emirates
4,855725,2023-09-05,2023-09-20,2023-09-27,250.00,Metric Ton,NaN,NaN,LCL,8711,...,False,NaN,NaN,NaN,IN,Chennai,DE,FRANKFURT AM MAIN,DE,"Frankfurt, Germany"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9893,941678,2024-03-05,2024-06-09,2024-06-16,30.00,Kilogram,NaN,NaN,LCL,NaN,...,False,NaN,IT,"Padua, Province of Padua, Italy",IT,Venice,CA,Toronto,CA,"Toronto, ON, Canada"
9894,941682,2024-03-05,2024-03-05,2024-03-12,28.23,Metric Ton,1.0,20' Standard,FCL,94-96,...,False,NaN,NaN,NaN,BR,Rio De Jan Eiro,SG,Singapore,NaN,NaN
9895,941683,2024-03-05,2024-06-09,2024-06-09,310.00,Kilogram,NaN,NaN,LCL,NaN,...,False,NaN,IT,"Padua, Province of Padua, Italy",IT,Venice,CA,Toronto,CA,"Toronto, ON, Canada"
9896,941687,2024-03-05,2024-04-06,2024-04-13,28.00,Metric Ton,1.0,20' Standard,FCL,50-63,...,False,NaN,ZA,"Cape Town City Centre, Cape Town, 8000, South ...",ZA,Cape Town,US,Charleston,NaN,NaN


In [51]:
air


,create_date,valid_from,valid_to,estimated_days,origin_country,origin_name,destination_country,destination_name,shipping_type,load_type,load_amount,price_usd
0,2024-02-11,2024-02-01,2024-12-31,6.0,ZW,"Harare, Zimbabwe",MZ,"Maputo, Mozambique",RAIL_FCL,40' Standard,1,2430.00
1,2024-02-11,2024-02-01,2024-12-31,6.0,ZW,"Harare, Zimbabwe",MZ,"Maputo, Mozambique",RAIL_FCL,20' Standard,1,1850.00
2,2024-02-11,2024-01-01,2024-12-31,5.0,MZ,"Beira, Mozambique",ZW,"Harare, Zimbabwe",RAIL_FCL,40' Standard,1,1850.00
3,2024-02-11,2024-01-01,2024-12-31,5.0,MZ,"Beira, Mozambique",ZW,"Harare, Zimbabwe",RAIL_FCL,20' Standard,1,1180.00
4,2024-02-11,2024-01-01,2024-12-31,5.0,ZW,"Harare, Zimbabwe",MZ,"Beira, Mozambique",RAIL_FCL,20' Standard,1,1180.00
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2023-10-09,2023-10-10,2023-10-11,NaN,US,"Los Angeles, CA, USA",US,"Via Cabrillo-Marina, San Pedro, CA 90731, USA",LTL,mt,1,293.79
996,2023-10-09,2023-11-24,2023-11-28,NaN,GB,"Victoria Embankment Gardens, Villiers St, Lond...",GB,"Cheshire, UK",LTL,mt,1,118.70
997,2023-10-09,2023-10-10,2023-10-12,NaN,US,"Los Angeles, CA, USA",US,"Via Cabrillo-Marina, San Pedro, CA 90731, USA",LTL,mt,1,135.94
998,2023-10-09,2023-11-29,2023-12-01,NaN,GB,"Victoria Embankment Gardens, Villiers St, Lond...",GB,"Cheshire, UK",LTL,mt,1,118.70


In [52]:
rates_air_renamed = air.rename(columns={
    'origin_country': 'port_from_country',
    'destination_country': 'port_to_country',
    'load_type': 'container_type'
})

# Объединение данных о запросах с тарифами для авиаперевозок.
# В реальной ситуации потребуется учитывать периоды действия тарифов, что может потребовать дополнительной обработки.
merged_leads_air = pd.merge(
    leads,
    rates_air_renamed,
    how='inner',
    on=['port_from_country', 'port_to_country', 'container_type']
)

merged_leads_air.head()


,request_id,request_date,valid_date_from,valid_date_to,weight,weight_unit,amount,container_type,shipping_type_x,hs_code,...,city_to,create_date,valid_from,valid_to,estimated_days,origin_name,destination_name,shipping_type_y,load_amount,price_usd
0,867146,2023-09-29,2023-10-01,2023-10-08,25.00,Metric Ton,1.0,40' Standard,FCL,94-96,...,NaN,2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Madrid, Spain",RAIL_FCL,1,9900.0
1,881615,2023-11-01,2023-11-30,2023-12-07,24.00,Metric Ton,1.0,40' Standard,FCL,392062,...,NaN,2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Madrid, Spain",RAIL_FCL,1,9900.0
2,903961,2024-01-02,2024-01-10,2024-01-17,10.00,Metric Ton,1.0,40' Standard,FCL,761699,...,"43460 Alcover, Tarragona, Spain",2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Madrid, Spain",RAIL_FCL,1,9900.0
3,916430,2024-01-24,2024-01-31,2024-02-07,10.46,Metric Ton,1.0,40' Standard,FCL,NaN,...,"Valencia, Spain",2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Madrid, Spain",RAIL_FCL,1,9900.0
4,916434,2024-01-24,2024-02-21,2024-02-28,10.46,Metric Ton,1.0,40' Standard,FCL,NaN,...,"Valencia, Spain",2023-11-28,2023-11-28,2023-12-31,NaN,"Wuhan, Hubei, China","Madrid, Spain",RAIL_FCL,1,9900.0


In [53]:
merged_leads_air['price_usd']

0     9900.0
1     9900.0
2     9900.0
3     9900.0
4     9900.0
5     9900.0
6     9900.0
7     9900.0
8     9450.0
9     9450.0
10    9450.0
11    9450.0
12    7450.0
13    7450.0
14    7450.0
15    8250.0
16    8250.0
17    9750.0
18    9750.0
19    9750.0
Name: price_usd, dtype: float64

In [54]:
# Подготовка и объединение данных о запросах на перевозку с тарифами для наземных перевозок.
rates_land_renamed = land.rename(columns={
    'origin_country': 'port_from_country',
    'destination_country': 'port_to_country',
    'load_type': 'container_type'
})
merged_leads_land = pd.merge(
    leads,
    rates_land_renamed,
    how='left',
    on=['port_from_country', 'port_to_country', 'container_type']
)

# Подготовка и объединение данных о запросах на перевозку с тарифами для морских перевозок.
rates_sea_renamed = sea.rename(columns={
    'origin_country': 'port_from_country',
    'destination_country': 'port_to_country',
    'load_type': 'container_type'
})
merged_leads_sea = pd.merge(
    leads,
    rates_sea_renamed,
    how='left',
    on=['port_from_country', 'port_to_country', 'container_type']
)



In [57]:
merged_leads_land['price_usd'].describe()


count    3161.000000
mean     2605.406478
std       950.939758
min        80.000000
25%      2050.000000
50%      2383.000000
75%      2950.000000
max      9394.000000
Name: price_usd, dtype: float64

In [58]:
merged_leads_sea['price_usd'].describe()


count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: price_usd, dtype: float64